## Exercises

- create a list of 3x3 matrices with random integer coefficients
- what is the probability that such a matrix is invertible if the coefficients are 0 and 1 ?
- use ```np.linalg.eigvals``` to find the biggest eigenvalue for the matrices

---

- make a [van der monde matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix)
with first row a random list of 5 real numbers
- calculate its determinant and verify the formula



In [4]:
import numpy as np
from numpy.random import randint

#?randint

```Docstring:
randint(low, high=None, size=None, dtype=int)

Return random integers from `low` (inclusive) to `high` (exclusive).

Return random integers from the "discrete uniform" distribution of
the specified dtype in the "half-open" interval [`low`, `high`). If
`high` is None (the default), then results are from [0, `low`).
```


In [7]:
randint(0,2,9)

array([1, 1, 1, 1, 0, 1, 1, 1, 0])

In [8]:
randint(0,2,9).reshape(3,3)

array([[0, 0, 1],
       [1, 0, 0],
       [1, 1, 1]])

In [9]:
from numpy.linalg import det

# Calculating the probability by simulation

this is easiest as it's just one line of code

we can calculate it exactly as there are only $2^9 = 512$
matrices 

In [12]:
[det(randint(0,2,9).reshape(3,3)) for k in range(10)]

[0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0]

# This works

but you should **never** test for a value of a floating point number

``` x == 0.0 ``` is **bad**

you should test for being in an interval

``` abs(x) < 0.1 ``` is **good**

In [13]:
[det(randint(0,2,9).reshape(3,3)) for k in range(10)].count(0.0)

6

In [143]:

num_tries =  10**5
X = [ 1 for k in range(num_tries) if abs(det(randint(0,2,9).reshape(3,3))) < 0.5 ]
len(X)/num_tries

0.66083

In [144]:
%%timeit

num_tries =  10**4
X = [ 1 for k in range(num_tries) if abs(det(randint(0,2,9).reshape(3,3))) < 0.5 ]
len(X)/num_tries

249 ms ± 26 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# This....

Might be faster (apparentl 4 times) but uses much more memory.
The reason is probably because ```reshape``` is called 
- $10^4$ times in the first solution
- once in the second

There is an extra **lookup** cost because Python has to find the method ```reshape``` each time.

---


There are two types of program
depending on what the limitations are

- processor bound
- I/O (memory bound) bound

There are tricks to get round these:

- When it's processor bound  like parellisation/concurrency.
- When it's I/O bound there is caching/memory mapped files.

Modern computers have a **lot** of memory and we hardly ever run out of it.


In [117]:
MM = randint(0,2,9*num_tries).reshape(3,3,-1)

In [118]:
MM[:,:,0]

array([[1, 1, 1],
       [1, 0, 0],
       [1, 0, 1]])

In [145]:
%%timeit

num_tries =  10**4
MM = randint(0,2,9*num_tries).reshape(3,3,-1)
X = [ 1 for k in range(num_tries) if abs(det(MM[:,:,k])) < 0.5 ]

65 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [125]:
len(X)

66203

In [141]:
num_tries

100000

---

# Calculating the probability exactly

we can do this by enumerating all the matrices

- this can be done using [recursion](https://realpython.com/python-thinking-recursively/#:~:text=Recursive%20Functions%20in%20Python,-Now%20that%20we&text=A%20recursive%20function%20is%20a,met%20to%20return%20a%20result.)
- but I'll do it using loops



---

This was my first attempt at making a list of all the possibilities.

It doesn't work because ```L``` is a list of lists 
so when I try to copy the elements using ```L[:]```
- I don't get a **real** copy of the lists 
- I just get a copy of **references** to the lists

[shallow vs deep copy](https://betterprogramming.pub/shallow-copy-vs-deep-copy-in-python-357e5f502bf9?gi=a13c5af07d2a#:~:text=In%20Python%2C%20a%20shallow%20copy,copies%20of%20the%20child%20objects.)

In [32]:
L = [[0],[1]]

for k in range(1):
    M = []
    for dd in [0,1]:
        tmp = L[:]
        print('>',L)
        for x in tmp:
            x.append(dd)
        print(tmp)
        M.extend(tmp)
        print(M)
    L = M
        
    

> [[0], [1]]
[[0, 0], [1, 0]]
[[0, 0], [1, 0]]
> [[0, 0], [1, 0]]
[[0, 0, 1], [1, 0, 1]]
[[0, 0, 1], [1, 0, 1], [0, 0, 1], [1, 0, 1]]


In [33]:
import copy

In [35]:
L = [[0],[1]]

for k in range(1):
    M = []
    for dd in [0,1]:
        tmp = copy.deepcopy(L)
        print('>',L)
        for x in tmp:
            x.append(dd)
        print(tmp)
        M.extend(tmp)
        print(M)
    L = M
        

> [[0], [1]]
[[0, 0], [1, 0]]
[[0, 0], [1, 0]]
> [[0], [1]]
[[0, 1], [1, 1]]
[[0, 0], [1, 0], [0, 1], [1, 1]]


In [50]:
L = [[0],[1]]

for k in range(8):
    # make a list to hold intermediate results
    M = []
    # add either 0 or 1 to the lists in L
    for dd in [0,1]:
        tmp = copy.deepcopy(L)
        # tmp = [x[:] for x in L] should work too
        for x in tmp:
            x.append(dd)
        M.extend(tmp)
        
    L = M
        

# Doing the same with ndarray

It took me a long time to get it to work
but it's a good exercise in **slicing**.

In [209]:
M = np.array([0,1])
for k in range(2):
    n = 2**(k+1)
    tt = np.ones((k+2,2*n))
    tt[:-1,n:] = tt[:-1,:n] = M
    tt[-1,:n] = 0
    M = tt
    

In [254]:
N = 3
M = np.ones((N,2**N))
M[0,0]= 0
for k in range(1,N):
    nn = 2**k
    M[:k,nn:2*nn]  =  M[:k,:nn]
    M[k,:nn] = 0
    
    
    
M


array([[0., 1., 0., 1., 0., 1., 0., 1.],
       [0., 0., 1., 1., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1., 1., 1., 1.]])


---

# So here is a trick


```[0, 0, 1, 1, 1, 1, 1, 1, 1]```

is the represenation of 127 in binary 

I'll use this to give another solution below

In [54]:
len(L), [ x for x in reversed(L[127]) ]

(512, [0, 0, 1, 1, 1, 1, 1, 1, 1])

In [56]:
DD = [ det(np.array(x).reshape(3,3)) for x in L]

In [59]:
len([1 for x in DD if abs(x) < .2])/512

0.66015625

---

# Using binary expansions

Technically this is a **trick**
programmers nowdays don't like using tricks 
with binary expansions or do they

Watch this sometime to see [Inverse square root Quake 3](https://youtu.be/p8u_k2LIZyo)


There is an easy algorithm that you should know
for writing a number $x$ in base $b$.

## Exo 

Modify my function ```f``` to write $x$ in base $5$



In [64]:
def f(x, N=3):
    ```calculate the fixed width binary expansion of x```
    L = []
    for k in range(N):
        L.append(x % 2)
        x = x // 2
    return L

N = 9
L = [ f(x,N=N) for x in range(2**N)]


In [65]:
DD = [ det(np.array(x).reshape(3,3)) for x in L]
len([1 for x in DD if abs(x) < .2])/512

0.66015625

# Matrices over  $F_2$


We talked about this in class and I said it should be easier 
**you should be able to check this**.

[read this](https://math.stackexchange.com/questions/2155710/number-of-non-singular-matrices-over-a-finite-field-of-order-2)

---

Here field of order 2 is F2 = Z2 = {0,1}. 
The result is actually the number of elements in the General Linear Group GL3(F2) or GL3(Z2).

To count the number of non-singular matrices of order 3 with 0 and 1 as its elements only, we have to make sure that all the rows are linearly independent and non-zero.

For the first row we have (23–1) choices.

For the second row we have (23–1)–1 = (23–2) choices. Because we cannot count the vector already has been used in first row.

For the third row we have (23–1)–2–1 = (23–22) choices. Because we have to omit 2 vectors from the count that already have been used in first and second row. And we have to omit one more vector that can be the linear combination of the first and second rows.


In [256]:
! ../.g

[master 5128655] web
 1 file changed, 9 insertions(+), 5 deletions(-)
Counting objects: 4, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 458 bytes | 458.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/macbuse/macbuse.github.io.git
   608aa35..5128655  master -> master


---

# Vandermonde

## I'm not doing it with random numbers 

- because I wanted to be sure the code worked
- you can (very easily)  change the code to make it work for **random** numbers

In [129]:
num_pts = 5

#X = np.random.random(num_pts)
X = np.arange(1,num_pts+1)
V = np.ones((num_pts,num_pts))

In [79]:
V

array([[1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1.]])

you can do it like this

In [84]:
for k in range(1,num_pts):
    V[k] = X**k
V

array([[  1.,   1.,   1.,   1.,   1.],
       [  1.,   2.,   3.,   4.,   5.],
       [  1.,   4.,   9.,  16.,  25.],
       [  1.,   8.,  27.,  64., 125.],
       [  1.,  16.,  81., 256., 625.]])

or like this which might be slightly cheaper computationally

In [130]:
V[1] = X
for k in range(2,5):
    V[k] = V[k-1]*X
V

array([[  1.,   1.,   1.,   1.,   1.],
       [  1.,   2.,   3.,   4.,   5.],
       [  1.,   4.,   9.,  16.,  25.],
       [  1.,   8.,  27.,  64., 125.],
       [  1.,  16.,  81., 256., 625.]])

---

# Verifying the formula

$\det(V) = \prod (x_i - x_j)$

- we need all the differences $x_i - x_j$

- it's actually easier to make an anti symmetric matrix $a_{i,j}$ with
$a_{ij} = x_i - x_j$

In [131]:
D = np.ones((num_pts,num_pts))
for k in range(num_pts):
    # make a row
    D[k] = X - X[k]

In [132]:
D

array([[ 0.,  1.,  2.,  3.,  4.],
       [-1.,  0.,  1.,  2.,  3.],
       [-2., -1.,  0.,  1.,  2.],
       [-3., -2., -1.,  0.,  1.],
       [-4., -3., -2., -1.,  0.]])

In [101]:
upper = [D[i,i+1:] for i in range(num_pts-1)]
upper 

[array([1., 2., 3., 4.]), array([1., 2., 3.]), array([1., 2.]), array([1.])]

?numpy.prod

Signature:
numpy.prod(
    a,
    axis=None,
    dtype=None,
    out=None,
    keepdims=<no value>,
    initial=<no value>,
    where=<no value>,
)
Docstring:
Return the product of array elements over a given axis.

In [102]:
[ np.prod(x) for x in upper]

[24.0, 6.0, 2.0, 1.0]

In [103]:
np.prod([ np.prod(x) for x in upper])

288.0

In [105]:
np.linalg.det(V)

-287.9999999999967

 you should be able to fix the sign 

---

# That's all folks